In [1]:
import os
import gc
import cv2
import torch
import numpy as np
import torchsummary

from sklearn.metrics import jaccard_score

from matplotlib import pyplot as plt
import segmentation_models_pytorch as smp
from torchvision import transforms, utils

%matplotlib inline

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
def preprocess_image(img: np.ndarray, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], resized = (224, 224)) -> torch.Tensor:
  
  preprocessing = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean, std),
                                      ])
  return preprocessing(img.copy()).unsqueeze(0)

In [4]:
def mask_img_to_mask(mask_path, bg_path):
    
    gt_mask = cv2.cvtColor(cv2.imread(mask_path), cv2.COLOR_BGR2GRAY)
    gt_bg   = cv2.cvtColor(cv2.imread(bg_path), cv2.COLOR_BGR2GRAY)
    
    tumor = (gt_mask == 52).astype(np.uint8).reshape(gt_mask.shape[0], gt_mask.shape[1], 1)
    stroma = (gt_mask == 94).astype(np.uint8).reshape(gt_mask.shape[0], gt_mask.shape[1], 1)
    normal = (gt_mask == 162).astype(np.uint8).reshape(gt_mask.shape[0], gt_mask.shape[1], 1)
    bg = (gt_bg/255).astype(np.uint8).reshape(gt_mask.shape[0], gt_mask.shape[1], 1)
    
    mask = np.concatenate((tumor, stroma, normal, bg), axis=2)
    
    return mask

In [5]:
def calculate_slice_bboxes(
    image_height: int,
    image_width: int,
    slice_height: int = 512,
    slice_width: int = 512,
    overlap_height_ratio: float = 0.2,
    overlap_width_ratio: float = 0.2,
) -> list[list[int]]:
    """
    Given the height and width of an image, calculates how to divide the image into
    overlapping slices according to the height and width provided. These slices are returned
    as bounding boxes in xyxy format.

    :param image_height: Height of the original image.
    :param image_width: Width of the original image.
    :param slice_height: Height of each slice
    :param slice_width: Width of each slice
    :param overlap_height_ratio: Fractional overlap in height of each slice (e.g. an overlap of 0.2 for a slice of size 100 yields an overlap of 20 pixels)
    :param overlap_width_ratio: Fractional overlap in width of each slice (e.g. an overlap of 0.2 for a slice of size 100 yields an overlap of 20 pixels)
    :return: a list of bounding boxes in xyxy format
    """

    slice_bboxes = []
    y_max = y_min = 0
    y_overlap = int(overlap_height_ratio * slice_height)
    x_overlap = int(overlap_width_ratio * slice_width)
    while y_max < image_height:
        x_min = x_max = 0
        y_max = y_min + slice_height
        while x_max < image_width:
            x_max = x_min + slice_width
            if y_max > image_height or x_max > image_width:
                xmax = min(image_width, x_max)
                ymax = min(image_height, y_max)
                xmin = max(0, xmax - slice_width)
                ymin = max(0, ymax - slice_height)
                slice_bboxes.append([xmin, ymin, xmax, ymax])
            else:
                slice_bboxes.append([x_min, y_min, x_max, y_max])
            x_min = x_max - x_overlap
        y_min = y_max - y_overlap
    return slice_bboxes

In [6]:
def img_resize(img, mask, factor):
    
    img_height, img_width, channels = img.shape
    
    #print(img.shape)
    
    new_height = int(factor * np.round(img_height/factor))
    new_width = int(factor * np.round(img_width/factor))
    
    img = cv2.resize(img, (max(new_width, 1), max(new_height, 1)))
    mask = cv2.resize(mask, (max(new_width, 1), max(new_height, 1)))
    
    #print(img.shape)
    
    slice_boxes = calculate_slice_bboxes(new_height, new_width, factor, factor, 0, 0)
    
    return img, mask, slice_boxes

In [7]:
def predict_mask(img):
    
    image_tensor = preprocess_image(img,)
    
    pred_mask = model(image_tensor.to(device))
    pred_mask = torch.nn.functional.softmax(pred_mask, dim=1)
    pred_mask = np.transpose(pred_mask.squeeze(0).cpu().detach().numpy(), (1, 2, 0))
    
    return pred_mask

In [8]:
model = smp.DeepLabV3Plus(encoder_name='resnet50', classes=4, activation=None, encoder_weights=None, ).to(device)
model.load_state_dict(torch.load(f='models/deeplabv3plus_dJ_par_resnet50_01.pth', map_location=device))

<All keys matched successfully>

In [9]:
%%capture
model.eval()

In [22]:
v_path = 'dataset/2.validation/2.validation/'
valid_images = os.listdir('dataset/Dataset/2.validation/2.validation/img/')

t_path = 'dataset/Dataset/3.testing/3.testing/'
test_images = os.listdir('desktop/Newcastle_University/11_FP_D/Dataset/Dataset/3.testing/3.testing/img/')

# Validation data

In [23]:
tumor_v_0  = []
stroma_v_0 = []
normal_v_0 = []

In [24]:
for i in valid_images:
    img_path  = v_path + 'img/' + str(i)
    mask_path = v_path + 'mask/' + str(i)
    bg_path   = v_path + 'background-mask/' + str(i)
    
    img     = cv2.imread(img_path)
    gt_mask = mask_img_to_mask(mask_path, bg_path)
    
    img, gt_mask, slice_boxes = img_resize(img, gt_mask, 224)
    pred = np.zeros(gt_mask.shape)
    
    for i in slice_boxes:
        #print(i)
        pred[i[1]:i[3], i[0]:i[2]] = predict_mask(img[i[1]:i[3], i[0]:i[2]])
    
    pred = pred.round().astype('uint8')
    
    tumor_v_0.append(jaccard_score(gt_mask[:, :, 0], pred[:, :, 0], average='macro', zero_division=1))
    stroma_v_0.append(jaccard_score(gt_mask[:, :, 1], pred[:, :, 1], average='macro', zero_division=1))
    normal_v_0.append(jaccard_score(gt_mask[:, :, 2], pred[:, :, 2], average='macro', zero_division=1))
    
    gc.collect()
    

In [25]:
print('Tumor', np.mean(tumor_v_0))
print('Stroma', np.mean(stroma_v_0))
print('Normal', np.mean(normal_v_0))
print('mIoU', (np.mean(tumor_v_0)+np.mean(stroma_v_0)+np.mean(normal_v_0))/3)

0.5819192156434992
0.38623145439061524
0.7843028032167065
0.5841511577502737


# Validation data with tumor and stroma considered as one

In [26]:
tumor_v_1  = []
normal_v_1 = []

In [27]:
for i in valid_images:
    img_path  = v_path + 'img/' + str(i)
    mask_path = v_path + 'mask/' + str(i)
    bg_path   = v_path + 'background-mask/' + str(i)
    
    img     = cv2.imread(img_path)
    gt_mask = mask_img_to_mask(mask_path, bg_path)
    
    img, gt_mask, slice_boxes = img_resize(img, gt_mask, 224)
    pred = np.zeros(gt_mask.shape)
    
    for i in slice_boxes:
        pred[i[1]:i[3], i[0]:i[2]] = predict_mask(img[i[1]:i[3], i[0]:i[2]])
    
    pred = pred.round().astype('uint8')
    
    gt_t = (gt_mask[:, :, 0] + gt_mask[:, :, 1])
    gt_t[gt_t>1] = 1
    pred_t = (pred[:, :, 0] + pred[:, :, 1])
    pred_t[pred_t>1] = 1
    
    tumor_v_1.append(jaccard_score(gt_t, pred_t, average='macro', zero_division=1))
    normal_v_1.append(jaccard_score(gt_mask[:, :, 2], pred[:, :, 2], average='macro', zero_division=1))
    
    gc.collect()

In [28]:
print('Tumor + Stroma', np.mean(tumor_v_1))
print('Normal', np.mean(normal_v_1))
print('mIoU', (np.mean(tumor_v_1)+np.mean(normal_v_1))/2)

0.7653811274326422
0.7843028032167065
0.7748419653246743


# Test data

In [29]:
tumor_v_0  = []
stroma_v_0 = []
normal_v_0 = []

In [30]:
for i in valid_images:
    img_path  = t_path + 'img/' + str(i)
    mask_path = t_path + 'mask/' + str(i)
    bg_path   = t_path + 'background-mask/' + str(i)
    
    img     = cv2.imread(img_path)
    gt_mask = mask_img_to_mask(mask_path, bg_path)
    
    img, gt_mask, slice_boxes = img_resize(img, gt_mask, 224)
    pred = np.zeros(gt_mask.shape)
    
    for i in slice_boxes:
        pred[i[1]:i[3], i[0]:i[2]] = predict_mask(img[i[1]:i[3], i[0]:i[2]])
    
    pred = pred.round().astype('uint8')
    
    tumor_v_0.append(jaccard_score(gt_mask[:, :, 0], pred[:, :, 0], average='macro', zero_division=1))
    stroma_v_0.append(jaccard_score(gt_mask[:, :, 1], pred[:, :, 1], average='macro', zero_division=1))
    normal_v_0.append(jaccard_score(gt_mask[:, :, 2], pred[:, :, 2], average='macro', zero_division=1))
    
    gc.collect()

In [31]:
print('Tumor', np.mean(tumor_v_0))
print('Stroma', np.mean(stroma_v_0))
print('Normal', np.mean(normal_v_0))
print('mIoU', (np.mean(tumor_v_0)+np.mean(stroma_v_0)+np.mean(normal_v_0))/3)

0.5276963141296049
0.41890617265571883
0.9416910281539431
0.6294311716464223


# Test data with tumor and stroma considered as one

In [32]:
tumor_v_1  = []
normal_v_1 = []

In [33]:
for i in valid_images:
    img_path  = t_path + 'img/' + str(i)
    mask_path = t_path + 'mask/' + str(i)
    bg_path   = t_path + 'background-mask/' + str(i)
    
    img     = cv2.imread(img_path)
    gt_mask = mask_img_to_mask(mask_path, bg_path)
    
    img, gt_mask, slice_boxes = img_resize(img, gt_mask, 224)
    pred = np.zeros(gt_mask.shape)
    
    for i in slice_boxes:
        pred[i[1]:i[3], i[0]:i[2]] = predict_mask(img[i[1]:i[3], i[0]:i[2]])
    
    pred = pred.round().astype('uint8')
    
    gt_t = (gt_mask[:, :, 0] + gt_mask[:, :, 1])
    gt_t[gt_t>1] = 1
    pred_t = (pred[:, :, 0] + pred[:, :, 1])
    pred_t[pred_t>1] = 1
    
    tumor_v_1.append(jaccard_score(gt_t, pred_t, average='macro', zero_division=1))
    normal_v_1.append(jaccard_score(gt_mask[:, :, 2], pred[:, :, 2], average='macro', zero_division=1))
    
    gc.collect()  

In [34]:
print('Tumor + Stroma', np.mean(tumor_v_1))
print('Normal', np.mean(normal_v_1))
print('mIoU', (np.mean(tumor_v_1)+np.mean(normal_v_1))/2)

0.8556801688992536
0.9416910281539431
0.8986855985265984
